In [1]:
%load_ext watermark


In [2]:
from downstream import dstream
from hstrat import hstrat
from hstrat import _auxiliary_lib as hstrat_aux
import pandas as pd
import scipy

In [3]:
%watermark -diwmuv -iv


Last updated: 2025-09-09T00:26:21.353451+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.31.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.8.0-1031-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

hstrat    : 1.20.10
pandas    : 2.2.3
scipy     : 1.14.1
downstream: 1.14.3

Watermark: 2.4.3



In [4]:
teeplot_subdir = "2025-09-07-shortcut-naive-comparison"
teeplot_subdir


'2025-09-07-shortcut-naive-comparison'

## Prep Data


In [5]:
reconstruction_error_data = pd.read_csv("https://osf.io/wc8h5/download")
reconstruction_error_data.head()


,Unnamed: 0,differentia_bitwidth,surface_size,fossil_interval,error,error_dropped_fossils
0,640,64,256,200.0,0.001109,0.003315
1,641,64,256,200.0,0.001064,0.002962
2,642,64,256,200.0,0.001669,0.004756
3,643,64,256,200.0,0.003648,0.005461
4,644,64,256,200.0,0.032434,0.022307


In [6]:
for group in reconstruction_error_data.groupby(["differentia_bitwidth", "surface_size", "fossil_interval"]):
    df = group[1]
    
    stat, p = scipy.stats.mannwhitneyu(
        df.iloc[1::2]["error_dropped_fossils"],  # shortcut
        df.iloc[::2]["error_dropped_fossils"],   # naive
        alternative="two-sided",
    )
    if p < 0.01:
        print(
            f"Significant difference in reconstruction error between shortcut and naive for differentia_bitwidth={group[0][0]}, surface_size={group[0][1]}, fossil_interval={group[0][2]}"
        )

Significant difference in reconstruction error between shortcut and naive for differentia_bitwidth=1, surface_size=32, fossil_interval=200.0
